In [67]:
import requests
import json
from datetime import date
from collections import defaultdict
from operator import itemgetter

In [35]:
# Get source data from api
def endpoint(page):
    return f"https://fixmyberlin.de/api/v1/sections?page={page}&page_size=200"

page = 1
has_next = True
data = []

while (has_next):
    print(f"Fetching {endpoint(page)}")
    resp = requests.get(endpoint(page)).json()
    data += resp["results"]
    print(f"Have {len(data)}")
    has_next = resp["next"] is not None
    page += 1
print(len(data))


Fetching https://fixmyberlin.de/api/v1/sections?page=1&page_size=200
Have 200
Fetching https://fixmyberlin.de/api/v1/sections?page=2&page_size=200
Have 400
Fetching https://fixmyberlin.de/api/v1/sections?page=3&page_size=200
Have 600
Fetching https://fixmyberlin.de/api/v1/sections?page=4&page_size=200
Have 800
Fetching https://fixmyberlin.de/api/v1/sections?page=5&page_size=200
Have 1000
Fetching https://fixmyberlin.de/api/v1/sections?page=6&page_size=200
Have 1200
Fetching https://fixmyberlin.de/api/v1/sections?page=7&page_size=200
Have 1400
Fetching https://fixmyberlin.de/api/v1/sections?page=8&page_size=200
Have 1600
Fetching https://fixmyberlin.de/api/v1/sections?page=9&page_size=200
Have 1800
Fetching https://fixmyberlin.de/api/v1/sections?page=10&page_size=200
Have 2000
Fetching https://fixmyberlin.de/api/v1/sections?page=11&page_size=200
Have 2200
Fetching https://fixmyberlin.de/api/v1/sections?page=12&page_size=200
Have 2400
Fetching https://fixmyberlin.de/api/v1/sections?page=

In [43]:
# Save data fetched
today = date.today().strftime("%y%m%d")
with open(f"hbi_{today}.json", "w") as f:
    json.dump(data, f)

In [84]:
# Define the format for data collection
by_borough = defaultdict(lambda: {
   "count": 0,
   "total_length": 0.0,
   "hbi_avg_product": 0.0
})

# Accumulate data by iterating over planning sections
for entry in data:
    target = by_borough[entry["borough"]]  # points at the target borough
    for side in entry["details"]:
        target["count"] += 1
        target["total_length"] += side["length"]
        target["hbi_avg_product"] += side["length"] * side["happy_bike_index"]

# Collect results in CSV format
out = "Bezirk,Planungsabschnitte,Länge (km),HBI (gewichteter Durchschnitt)\n"
for (name, borough) in sorted(by_borough.items(), key=itemgetter(0)):
    hbi_avg = borough["hbi_avg_product"] / borough["total_length"]
    length = borough["total_length"] / 1000.0
    count = borough["count"]
    out += f"{name},{count},{length:.2f},{hbi_avg:.2f}\n"


In [85]:
with open("by_borough.csv", "w") as f:
    f.write(out)